# 说明：
* 该文档为使用foundry的酶从头设计（M0255_1mg5.pdb）和酶部分扩散（7v11.pdb）的范例文件
* 跑一个设计（设计1个骨架，10个序列，10个结构），需要时间为1小时，不太建议在colab中运行
* 优化点：使用MyDrive存储权重文件，能加快环境安装速度，由接近10min缩短至3min
* 注意：由于 T4 gpu 的原因，强制关闭了 rf3 的Triton 加速
* 该文件酶设计（RFD3 + MPNN + RF3）的事例文件
* 1 和 1.1 为两个设计方向，不能在一个运行时中都运行



In [1]:
# @title 环境安装
import os
import shutil
from google.colab import drive

# ==========================================
# 1. 挂载 Google Drive
# ==========================================
if not os.path.exists('/content/drive'):
    print("正在挂载 Google Drive...")
    drive.mount('/content/drive')

# ==========================================
# 2. 配置路径
# ==========================================
# 请确保这里是你 Drive 中存放权重的真实路径
DRIVE_WEIGHTS_DIR = "/content/drive/MyDrive/Foundry_Weights"
LOCAL_WEIGHTS_DIR = os.path.expanduser("~/.foundry")

# 确保 Drive 文件夹存在
os.makedirs(DRIVE_WEIGHTS_DIR, exist_ok=True)

# ==========================================
# 3. 智能处理本地文件（核心保护逻辑）
# ==========================================

# 如果本地有一个真实的文件夹（这意味着是你之前下载好的）
if os.path.exists(LOCAL_WEIGHTS_DIR) and not os.path.islink(LOCAL_WEIGHTS_DIR):
    print(f"检测到本地有已下载的权重: {LOCAL_WEIGHTS_DIR}")

    # 检查 Drive 里是不是空的
    drive_files = os.listdir(DRIVE_WEIGHTS_DIR)

    if not drive_files:
        # 情况 A: Drive 是空的 -> 必须把本地数据搬过去保存！
        print(f"Drive 目录为空，正在将本地数据备份到 Drive...")
        print("（这可能需要几分钟，请耐心等待拷贝完成）")
        # 复制文件
        for item in os.listdir(LOCAL_WEIGHTS_DIR):
            s = os.path.join(LOCAL_WEIGHTS_DIR, item)
            d = os.path.join(DRIVE_WEIGHTS_DIR, item)
            if os.path.isdir(s):
                shutil.copytree(s, d, dirs_exist_ok=True)
            else:
                shutil.copy2(s, d)
        print("✅ 备份完成！数据已安全存入 Google Drive。")
    else:
        # 情况 B: Drive 里已经有文件了 -> 本地的是重复副本
        print(f"✅ 检测到 Drive 中已有数据 ({len(drive_files)} 个文件/文件夹)。")
        print("本地文件与 Drive 重复。")

    # 只有确认 Drive 里有数据后，才移除本地文件夹，为建立链接腾出位置
    print("正在移除本地临时文件夹以建立云端链接...")
    shutil.rmtree(LOCAL_WEIGHTS_DIR)

# ==========================================
# 4. 建立软链接
# ==========================================
if not os.path.exists(LOCAL_WEIGHTS_DIR):
    os.symlink(DRIVE_WEIGHTS_DIR, LOCAL_WEIGHTS_DIR)
    print(f"🔗 链接建立成功！程序现在直接读取 Drive 数据：\n {LOCAL_WEIGHTS_DIR} -> {DRIVE_WEIGHTS_DIR}")
else:
    if os.path.islink(LOCAL_WEIGHTS_DIR):
        print("🔗 软链接已就绪。")

# ==========================================
# 5. 后续安装与验证
# ==========================================
os.environ['CCD_MIRROR_PATH'] = ''
os.environ['PDB_MIRROR_PATH'] = ''

if not os.path.isfile("FOUNDRY_READY"):
    print("正在配置环境...")
    os.system("pip uninstall -y torchvision")
    os.system("pip install -q 'rc-foundry[all]'")
    os.system("touch FOUNDRY_READY")

print("-" * 30)
print("验证模型权重...")
# 这一步应该瞬间完成
os.system("foundry install rfd3 ligandmpnn rf3 proteinmpnn")
print("🎉 全部完成！")

# ==============================================
import warnings
warnings.filterwarnings('ignore', module='atomworks')

# Shared utilities for visualization (from AtomWorks)
from atomworks.io.utils.visualize import view

正在挂载 Google Drive...
Mounted at /content/drive
🔗 链接建立成功！程序现在直接读取 Drive 数据：
 /root/.foundry -> /content/drive/MyDrive/Foundry_Weights
正在配置环境...
------------------------------
验证模型权重...
🎉 全部完成！


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  


In [2]:
# @title 上传输入文件
import os
import shutil
from google.colab import files
from pathlib import Path

# 1. 定义基础路径 (修复 base_dir 未定义的问题)
base_dir = Path(".") # 或者使用 Path("/content")
input_dir = base_dir / "01_input"

# 2. 创建目录 (使用 pathlib 的 mkdir)
# parents=True 允许创建多级目录, exist_ok=True 防止目录已存在报错
input_dir.mkdir(parents=True, exist_ok=True)

# @markdown 是否清理旧的输入文件？
clear_previous = True # @param {type:"boolean"}

if clear_previous:
    print(f"🧹 正在清理旧文件: {input_dir}")
    # 遍历并删除文件，保留子文件夹(如果需要删除子文件夹需改用 shutil.rmtree)
    for item in input_dir.iterdir():
        if item.is_file():
            item.unlink()
        elif item.is_dir():
            shutil.rmtree(item)

print(f"📂 请上传文件到: {input_dir.resolve()}")
uploaded = files.upload()

for filename in uploaded.keys():
    # 获取上传后的临时文件路径 (Colab 默认上传到当前工作区根目录)
    src = base_dir / filename
    dst = input_dir / filename

    # 移动文件
    shutil.move(str(src), str(dst))
    print(f"✅ 已保存: {dst}")

🧹 正在清理旧文件: 01_input
📂 请上传文件到: /content/01_input


Saving 7v11.pdb to 7v11.pdb
✅ 已保存: 01_input/7v11.pdb


In [3]:
# @title 导包
import os
import csv
import json
import numpy as np
import pandas as pd
import torch

# 引入必要的库
from lightning.fabric import seed_everything
from biotite.sequence import ProteinSequence
from biotite.structure import get_residue_starts, rmsd, superimpose
from atomworks.io.utils.io_utils import to_cif_file
from atomworks.constants import PROTEIN_BACKBONE_ATOM_NAMES

# 引入 Foundry 模型
from rfd3.engine import RFD3InferenceConfig, RFD3InferenceEngine
from mpnn.inference_engines.mpnn import MPNNInferenceEngine
from rf3.inference_engines.rf3 import RF3InferenceEngine
from rf3.utils.inference import InferenceInput
import rf3.model.layers.attention  # 用于强制关闭加速

INFO:foundry:cuEquivariance is available and will be used.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  
DEBUG:transforms:Debug mode is on
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  


In [4]:
# @title 全局设置与目录创建
# ==========================================
# 0. 全局设置与目录创建
# ==========================================
seed_everything(0)

# 定义输出目录
DIR_RFD3 = "02_outputs_rfd3"
DIR_MPNN = "03_outputs_mpnn"
DIR_RF3  = "04_outputs_rf3"

for d in [DIR_RFD3, DIR_MPNN, DIR_RF3]:
    os.makedirs(d, exist_ok=True)


# 输入文件路径 (请确保该文件存在)
# 请根据您实际上传的文件路径修改这里，例如 "./M0255_1mg5.pdb"

INPUT_PDB = os.path.abspath("./01_input/7v11.pdb")

print(f"\n🚀 流程开始... 使用输入: {INPUT_PDB}")

INFO: Seed set to 0
INFO:lightning.fabric.utilities.seed:Seed set to 0



🚀 流程开始... 使用输入: /content/01_input/7v11.pdb


In [ ]:
# @title 1. RFdiffusion3: 从头生成
# ==========================================
# 1. RFdiffusion3: 骨架生成
# ==========================================

print("正在运行 RFdiffusion3 生成骨架...")
rfd3_config = RFD3InferenceConfig(
    specification={
        "input": INPUT_PDB,
        "ligand": "NAI,ACT",
        "unindex": "A108,A139,A152,A156",
        "length": "180-200",
        "select_fixed_atoms": {
            "A108": "ND2,CG",
            "A139": "OG,CB,CA",
            "A152": "OH,CZ",
            "A156": "NZ,CE,CD",
            "ACT": "OXT",
            "NAI": ""
        },
        "extra": {}
    },

    diffusion_batch_size=1,
    low_memory_mode=True
)

rfd3_engine = RFD3InferenceEngine(**rfd3_config)

rfd3_outputs_dict = rfd3_engine.run(inputs=None, out_dir=None, n_batches=1)

# 保存 RFD3 结果和 CSV
rfd3_records = []
rfd3_backbones = {}

csv_rfd3_path = os.path.join(DIR_RFD3, 'rfd3_results.csv')
print(f"  -> 正在保存 RFD3 结果至 {csv_rfd3_path} ...")

with open(csv_rfd3_path, 'w', newline='') as f:
    writer = csv.writer(f)
    # 增加 Seed 和 Metadata 列，记录生成参数
    writer.writerow(['design_id', 'length', 'seed', 'metadata', 'file_path'])

    for batch_key, structure_list in rfd3_outputs_dict.items():
        for i, rfd3_obj in enumerate(structure_list):
            design_id = f"backbone_{batch_key}_{i}"
            file_name = f"{design_id}.cif"
            file_path = os.path.join(DIR_RFD3, file_name)

            # 保存 CIF 结构
            to_cif_file(rfd3_obj.atom_array, file_path)

            # 缓存骨架用于后续验证
            rfd3_backbones[design_id] = rfd3_obj.atom_array
            rfd3_records.append(design_id)

            # 提取元数据
            length = len(get_residue_starts(rfd3_obj.atom_array))
            # 尝试从 metadata 中提取 seed，如果没有则为空
            seed_val = rfd3_obj.metadata.get('seed', 'N/A')
            # 将 metadata 字典转为字符串保存，方便后续查阅
            meta_str = json.dumps(rfd3_obj.metadata) if rfd3_obj.metadata else "{}"

            writer.writerow([design_id, length, seed_val, meta_str, file_path])

print(f" RFD3 完成。生成了 {len(rfd3_records)} 个骨架。")

INFO:rfd3.engine:[rank: 0] Low memory mode enabled.


正在运行 RFdiffusion3 生成骨架...


INFO:rfd3.engine:[rank: 0] Finished inference batch in 136.83 seconds.


  -> 正在保存 RFD3 结果至 02_outputs_rfd3/rfd3_results.csv ...
 RFD3 完成。生成了 1 个骨架。


In [11]:
# @title 1.1 RFD3 部分扩散
from lightning.fabric import seed_everything
from rfd3.engine import RFD3InferenceConfig, RFD3InferenceEngine

# Set seed for reproducibility
seed_everything(0)

# Configure RFD3 inference
config = RFD3InferenceConfig(
    specification={
        "input": INPUT_PDB,
        "ligand": "OQO",
        "partial_t": 15.0,
        "contig": "A431",
        "unindex": "A572-573",
        "select_fixed_atoms": {
            "A431": "TIP",
            "A572": "BKBN",
            "A573": "BKBN"
        },

        "extra": {}
    },
    diffusion_batch_size=1,
    low_memory_mode=True,
    ckpt_path="drive/MyDrive/Foundry_Weights/checkpoints/rfd3_latest.ckpt"
)

rfd3_engine = RFD3InferenceEngine(**config)
rfd3_outputs_dict = rfd3_engine.run(inputs=None, out_dir=None, n_batches=1)
# 保存 RFD3 结果和 CSV
rfd3_records = []
rfd3_backbones = {}

csv_rfd3_path = os.path.join(DIR_RFD3, 'rfd3_results.csv')
print(f"  -> 正在保存 RFD3 结果至 {csv_rfd3_path} ...")

with open(csv_rfd3_path, 'w', newline='') as f:
    writer = csv.writer(f)
    # 增加 Seed 和 Metadata 列，记录生成参数
    writer.writerow(['design_id', 'length', 'seed', 'metadata', 'file_path'])

    for batch_key, structure_list in rfd3_outputs_dict.items():
        for i, rfd3_obj in enumerate(structure_list):
            design_id = f"backbone_{batch_key}_{i}"
            file_name = f"{design_id}.cif"
            file_path = os.path.join(DIR_RFD3, file_name)

            # 保存 CIF 结构
            to_cif_file(rfd3_obj.atom_array, file_path)

            # 缓存骨架用于后续验证
            rfd3_backbones[design_id] = rfd3_obj.atom_array
            rfd3_records.append(design_id)

            # 提取元数据
            length = len(get_residue_starts(rfd3_obj.atom_array))
            # 尝试从 metadata 中提取 seed，如果没有则为空
            seed_val = rfd3_obj.metadata.get('seed', 'N/A')
            # 将 metadata 字典转为字符串保存，方便后续查阅
            meta_str = json.dumps(rfd3_obj.metadata) if rfd3_obj.metadata else "{}"

            writer.writerow([design_id, length, seed_val, meta_str, file_path])

print(f" RFD3 完成。生成了 {len(rfd3_records)} 个骨架。")

INFO:rfd3.engine:[rank: 0] Finished inference batch in 1396.49 seconds.


In [13]:
# @title 2. LigandMPNN: 序列设计
# ==========================================
# 2. LigandMPNN: 序列设计
# ==========================================
print("正在运行 LigandMPNN 设计序列...")

# 准备输入：将生成的骨架转为 MPNN 输入格式
mpnn_input_configs = [{"batch_size": 10, "remove_waters": True} for _ in rfd3_records]
all_rfd3_structures = [s for sublist in rfd3_outputs_dict.values() for s in sublist]

mpnn_config = {
    "model_type": "ligand_mpnn",
    "is_legacy_weights": True,
    "out_directory": None,
    "write_structures": False,
    "write_fasta": False,
}

mpnn_engine = MPNNInferenceEngine(**mpnn_config)
mpnn_outputs = mpnn_engine.run(input_dicts=mpnn_input_configs, atom_arrays=[s.atom_array for s in all_rfd3_structures])

# 保存 MPNN 结果
csv_mpnn_path = os.path.join(DIR_MPNN, 'mpnn_results.csv')
mpnn_results_list = [] # 用于传递给 RF3

print(f"  -> 正在保存 MPNN 序列至 {DIR_MPNN} ...")

with open(csv_mpnn_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['seq_id', 'backbone_id', 'sequence', 'recovery', 'fasta_path'])

    for i, mpnn_obj in enumerate(mpnn_outputs):
        seq_id = f"seq_{i}"
        # 简单的索引映射 (假设 batch_size=10)
        backbone_idx = i // 10
        backbone_id = rfd3_records[backbone_idx] if backbone_idx < len(rfd3_records) else "unknown"

        # 提取序列 (只提取 Chain A，避开配体报错)
        target_chain = "A"
        protein_atoms = mpnn_obj.atom_array[mpnn_obj.atom_array.chain_id == target_chain]

        seq_1letter = ""
        if protein_atoms.array_length() > 0:
            try:
                res_starts = get_residue_starts(protein_atoms)
                seq_1letter = "".join([ProteinSequence.convert_letter_3to1(r) for r in protein_atoms.res_name[res_starts]])
            except KeyError:
                seq_1letter = "ERROR_UNKNOWN_RESIDUE"

        recovery = f"{mpnn_obj.output_dict.get('sequence_recovery', 0.0):.4f}"

        # 保存 FASTA
        fasta_name = f"{seq_id}.fasta"
        fasta_path = os.path.join(DIR_MPNN, fasta_name)
        with open(fasta_path, "w") as f_fa:
            f_fa.write(f">{seq_id} | backbone={backbone_id} | recovery={recovery}\n{seq_1letter}\n")

        writer.writerow([seq_id, backbone_id, seq_1letter, recovery, fasta_path])

        # 收集数据给 RF3
        mpnn_results_list.append({
            "seq_id": seq_id,
            "backbone_id": backbone_id,
            "atom_array": mpnn_obj.atom_array
        })

print(f"MPNN 完成。设计了 {len(mpnn_results_list)} 条序列。")


正在运行 LigandMPNN 设计序列...
  -> 正在保存 MPNN 序列至 03_outputs_mpnn ...
MPNN 完成。设计了 10 条序列。


In [14]:
# @title 3. RF3: 结构验证 & 筛选
# ==========================================
# 3. RF3: 结构验证 (重折叠) & 筛选
# ==========================================
print("正在运行 RF3 进行验证")

# 强制关闭 Triton 加速以防报错
rf3.model.layers.attention.SHOULD_USE_CUEQUIVARIANCE = False

rf3_engine = RF3InferenceEngine(ckpt_path='rf3', verbose=False)
rf3_engine._assign_override("model.use_cuequivariance", False)

validation_data = []

# 遍历所有 MPNN 设计的序列进行折叠
for idx, item in enumerate(mpnn_results_list):
    seq_id = item["seq_id"]
    backbone_id = item["backbone_id"]

    print(f"  -> 验证序列 {idx+1}/{len(mpnn_results_list)}: {seq_id}")

    # 构建输入
    rf3_input = InferenceInput.from_atom_array(item["atom_array"], example_id=seq_id)

    # 运行 RF3
    rf3_result_dict = rf3_engine.run(inputs=rf3_input)

    if seq_id in rf3_result_dict:
        rf3_prediction = rf3_result_dict[seq_id][0]

        # 保存预测结构
        pred_filename = f"validation_{seq_id}.cif"
        pred_path = os.path.join(DIR_RF3, pred_filename)
        to_cif_file(rf3_prediction.atom_array, pred_path)

        # --- 指标提取 ---
        summ = rf3_prediction.summary_confidences
        plddt = summ.get('overall_plddt', 0)
        ptm = summ.get('ptm', 0)
        has_clash = summ.get('has_clash', False)

        # 界面 PAE 计算
        pae_interaction = 999.0
        chain_pair_pae = summ.get('chain_pair_pae')
        if chain_pair_pae and len(chain_pair_pae) >= 2:
            try:
                vals = [x for x in [chain_pair_pae[0][1], chain_pair_pae[1][0]] if x is not None]
                if vals: pae_interaction = sum(vals) / len(vals)
            except: pass

        # --- RMSD 计算 (自洽性) ---
        rmsd_val = 999.0
        if backbone_id in rfd3_backbones:
            try:
                ref_atoms = rfd3_backbones[backbone_id]
                pred_atoms = rf3_prediction.atom_array

                # 仅使用 Chain A 的 N, CA, C (最稳健的筛选方式)
                target_atoms = ["N", "CA", "C"]
                ref_bb = ref_atoms[(ref_atoms.chain_id == "A") & np.isin(ref_atoms.atom_name, target_atoms)]
                pred_bb = pred_atoms[(pred_atoms.chain_id == "A") & np.isin(pred_atoms.atom_name, target_atoms)]

                # 长度对齐并叠加
                min_len = min(len(ref_bb), len(pred_bb))
                if min_len > 0:
                    pred_fitted, _ = superimpose(ref_bb[:min_len], pred_bb[:min_len])
                    rmsd_val = rmsd(ref_bb[:min_len], pred_fitted)
            except Exception as e:
                print(f"    ⚠️ RMSD calc failed for {seq_id}: {e}")

        validation_data.append({
            "seq_id": seq_id,
            "backbone_id": backbone_id,
            "plddt": plddt,
            "rmsd": rmsd_val,
            "pae_interaction": pae_interaction,
            "has_clash": has_clash,
            "ptm": ptm,
            "file_path": pred_path
        })

# 保存详细验证结果
df_val = pd.DataFrame(validation_data)
csv_rf3_path = os.path.join(DIR_RF3, 'rf3_results_enhanced.csv')
df_val.to_csv(csv_rf3_path, index=False)
print(f"✅ RF3 验证完成。详细报告已保存至 {csv_rf3_path}")


正在运行 RF3 进行验证
  -> 验证序列 1/10: seq_0


INFO:rf3.inference_engines.rf3:[rank: 0] Loading checkpoint from /content/drive/MyDrive/Foundry_Weights/checkpoints/rf3_foundry_01_24_latest_remapped.ckpt...
INFO:rf3.inference_engines.rf3:[rank: 0] Found 1 structures to predict!
INFO:rf3.inference_engines.rf3:[rank: 0] Predicting structure 1/1: seq_0


  -> 验证序列 2/10: seq_1


INFO:rf3.inference_engines.rf3:[rank: 0] Found 1 structures to predict!
INFO:rf3.inference_engines.rf3:[rank: 0] Predicting structure 1/1: seq_1


  -> 验证序列 3/10: seq_2


...

  -> 验证序列 9/10: seq_8


INFO:rf3.inference_engines.rf3:[rank: 0] Found 1 structures to predict!
INFO:rf3.inference_engines.rf3:[rank: 0] Predicting structure 1/1: seq_8


  -> 验证序列 10/10: seq_9


INFO:rf3.inference_engines.rf3:[rank: 0] Found 1 structures to predict!
INFO:rf3.inference_engines.rf3:[rank: 0] Predicting structure 1/1: seq_9


✅ RF3 验证完成。详细报告已保存至 04_outputs_rf3/rf3_results_enhanced.csv


In [26]:
# @title 4.自动筛选
# ==========================================
# 4. 自动筛选
# ==========================================
print("正在筛选最佳候选者...")

TH_RMSD = 2.0
# 处理 pLDDT 可能为 0-1 或 0-100 的情况
max_plddt = df_val['plddt'].max() if not df_val.empty else 0
TH_PLDDT = 0.8 if max_plddt <= 1.0 else 80.0

if not df_val.empty:
    df_final = df_val[
        (df_val['has_clash'] == False) &
        (df_val['rmsd'] < TH_RMSD) &
        (df_val['plddt'] > TH_PLDDT)
    ].copy()

    # 排序：优先低 RMSD，其次高 pLDDT
    df_final = df_final.sort_values(by=['rmsd', 'plddt'], ascending=[True, False])

    final_csv_path = os.path.join(DIR_RF3, 'final_candidates.csv')
    df_final.to_csv(final_csv_path, index=False)

    print(f" 筛选完成！")
    print(f"  - 总验证数: {len(df_val)}")
    print(f"  - 通过筛选: {len(df_final)}")
    print(f"  - 最终名单: {final_csv_path}")

    if len(df_final) > 0:
        print("\nTop 3 候选者:")
        print(df_final[['seq_id', 'rmsd', 'plddt']].head(3))
    else:
        print("\n没有序列通过严格筛选。建议：")
        print("  1. 检查输入 PDB 是否合理")
        print("  2. 在 MPNN 步骤增加 batch_size 生成更多序列")
        print("  3. 放宽 RMSD 阈值 (如 < 3.0)")
else:
    print("没有产生验证数据，流程可能在 RF3 阶段中断。")

正在筛选最佳候选者...
 筛选完成！
  - 总验证数: 10
  - 通过筛选: 0
  - 最终名单: 04_outputs_rf3/final_candidates.csv

没有序列通过严格筛选。建议：
  1. 检查输入 PDB 是否合理
  2. 在 MPNN 步骤增加 batch_size 生成更多序列
  3. 放宽 RMSD 阈值 (如 < 3.0)


In [20]:
# @title 可视化第一个结构
# 1. 获取字典里的第一个 key (即你的序列 ID)
first_key = next(iter(rf3_result_dict.keys()))

# 2. 获取该 ID 对应的预测列表中的第一个结果 (RF3Output 对象)
rf3_prediction = rf3_result_dict[first_key][0]

# 3. 现在可以访问 atom_array 并可视化了
print(f"正在查看序列: {first_key}")
view(rf3_prediction.atom_array)

正在查看序列: seq_9


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [24]:
# @title 打包
import os
import zipfile
from google.colab import files

def package_and_download_results(zip_filename="experiment_full_results.zip"):
    """
    自动打包 RFD3/MPNN/RF3 的输出目录及报告，并触发浏览器下载。
    """
    # 定义需要打包的目标列表 (文件或文件夹均可)
    targets_to_zip = [
        "01_input",
        "02_outputs_rfd3",
        "03_outputs_mpnn",
        "04_outputs_rf3"


    ]

    print(f"📦 正在创建压缩包: {zip_filename} ...")

    files_added = 0

    # 创建压缩文件
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for target in targets_to_zip:
            if os.path.exists(target):
                if os.path.isdir(target):
                    # 如果是文件夹，递归添加所有内容
                    print(f"  📂 添加目录: {target}")
                    for root, dirs, filenames in os.walk(target):
                        for filename in filenames:
                            file_path = os.path.join(root, filename)
                            # 保持压缩包内的目录结构
                            arcname = os.path.relpath(file_path, start=".")
                            zipf.write(file_path, arcname)
                            files_added += 1
                else:
                    # 如果是文件，直接添加
                    print(f"  📄 添加文件: {target}")
                    zipf.write(target, arcname=target)
                    files_added += 1
            else:
                print(f"  ⚠️ 跳过 (未找到): {target}")

    # 检查是否有文件被添加
    if files_added > 0:
        file_size_mb = os.path.getsize(zip_filename) / (1024 * 1024)
        print(f"\n✅ 压缩完成! (共 {files_added} 个文件, 大小: {file_size_mb:.2f} MB)")
        print("🚀 正在触发下载，请留意浏览器弹窗...")
        files.download(zip_filename)
    else:
        print("\n❌ 错误: 没有找到任何目标文件，无法下载。")

# --- 执行打包下载 ---
package_and_download_results()

📦 正在创建压缩包: experiment_full_results.zip ...
  📂 添加目录: 01_input
  📂 添加目录: 02_outputs_rfd3
  📂 添加目录: 03_outputs_mpnn
  📂 添加目录: 04_outputs_rf3

✅ 压缩完成! (共 26 个文件, 大小: 0.68 MB)
🚀 正在触发下载，请留意浏览器弹窗...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>